## SETUP

In [5]:
import time
import pandas as pd
# Android ADB
from subprocess import check_output,Popen
# Notebook
from IPython.display import display,clear_output
from tqdm.notebook import trange, tqdm
import configparser
import sys
import os
# internal
sys.path.append('./src')
import bot_core
import bot_perception
import port_scan

import importlib
[importlib.reload(lib) for lib in [bot_core,bot_perception,port_scan]]

config = configparser.ConfigParser()
# Read the config file
config.read('config.ini')
# Get the values from the config file
scrcpy_path=config['bot']['scrcpy_path']
static_adb=config['bot'].getboolean('static_adb_port')

device = port_scan.get_device()
if device is None:
    raise Exception("No device found!")
    
# Start Scrcpy once per restart
if 'started_scrcpy' not in vars():
    started_scrcpy=True
    proc = Popen([os.path.join(scrcpy_path,'scrcpy'),'-s',device], shell=True)
    time.sleep(3) # <-- sleep for 6''
    proc.terminate() # <-- terminate the process (Scrcpy window can be closed)

# Selected units in deck (order does not matter)
#sel_units= ['crystal.png','chemist.png','monkey.png','dryad.png','hunter.png']
sel_units= ['chemist.png','knight_statue.png','harlequin.png','dryad.png','demon_hunter.png']
bot_core.select_units(sel_units,show=False)
bot = bot_core.Bot(device)

Found ADB device! emulator-5554
Log file is already open


In [7]:
bot_core.select_units(sel_units,show=True)

['alchemist.png', 'archer.png', 'banner.png', 'blade_dancer.png', 'bruser.png', 'catapult.png', 'cauldron.png', 'chemist.png', 'clock.png', 'cold_mage.png', 'corsair.png', 'crystal.png', 'cultist.png', 'cultist_off.png', 'demo.png', 'demon_hunter.png', 'dryad.png', 'empty.png', 'engineer.png', 'executioner.png', 'fire_mage.png', 'flinger.png', 'gargoyle.png', 'harlequin.png', 'hex.png', 'hunter.png', 'inquisitor.png', 'ivy.png', 'knight_statue.png', 'lightning.png', 'meteor.png', 'mime.png', 'monkey.png', 'old', 'old_icon', 'poisoner.png', 'portal_keeper.png', 'portal_mage.png', 'priest.png', 'pyro.png', 'reaper.png', 'rogue.png', 'sentry.png', 'shaman.png', 'sharpshooter.png', 'sharpstone.png', 'summoner.png', 'thunder.png', 'time_keeper.png', 'trapper.png', 'unit_rank', 'vampire.png', 'wind_arch.png', 'witch_doc.png', 'zealot.png']
Chosen:
 ['cauldron.png', 'knight_statue.png', 'harlequin.png', 'dryad.png', 'zealot.png']


## RUN BOT

In [6]:
wait=0
combat = 0
grid_df =None
watch_ad = False
train_ai = False
# Main loop
while(1):
    output = bot.battle_screen(start=False)
    if output[1]=='fighting':
        watch_ad = True 
        wait = 0
        combat+=1
        if combat>50:
            bot.restart_RR()
            combat = 0
            continue
        battle_bar=trange(8)
        for i in battle_bar:
                time.sleep(0.2)
                # Upgrade units
                bot.mana_level([2,3,5],hero_power=True)
                # Spawn units
                bot.click(450,1360)
                # Try to merge units
                grid_df,unit_series,merge_series,df_groups,info = bot.try_merge(prev_grid=grid_df,merge_target='demon_hunter.png')
                clear_output()
                display([f'{combat}, {i+1}/8 {output[1]}, {info}', 'Average age: '+ str(grid_df['Age'].mean().round(2)),grid_df,unit_series,df_groups])
        # Wait until late stage in combat and if consistency is ok, not stagnate save all units for ML model
        if combat==25 and 5<grid_df['Age'].mean()<50 and train_ai:
            bot_perception.add_grid_to_dataset()
    elif output[1]=='home' and watch_ad:
        for i in range(3):
            bot.watch_ads()
        watch_ad = False
    else:
        combat=0
        output = bot.battle_screen(start=True,pve=True,floor=7) #(only 1,2,4,5,7,8,10 possible)
        wait+=1
        if wait>40:
            print('RESTARTING')
            bot.restart_RR(),
            
            wait=0
        print(output[1],'wait count:',wait)

['19, 2/8 fighting, Merging!',
 'Average age: 7.8',
    grid_pos               unit  probability  rank  rank_prob   Age
 0    [0, 0]   demon_hunter.png        0.829     5   0.999999   5.0
 1    [0, 1]        chemist.png        0.925     1   1.000000  13.0
 2    [0, 2]          dryad.png        0.808     3   1.000000   0.0
 3    [0, 3]  knight_statue.png        0.916     2   1.000000   5.0
 4    [0, 4]          dryad.png        0.894     2   0.999999  15.0
 5    [1, 0]        chemist.png        0.846     2   0.999999  13.0
 6    [1, 1]  knight_statue.png        0.912     2   0.999999   4.0
 7    [1, 2]          empty.png        0.000     0   0.578992   5.0
 8    [1, 3]          dryad.png        0.853     2   0.999970  25.0
 9    [1, 4]  knight_statue.png        0.930     1   1.000000   4.0
 10   [2, 0]          empty.png        0.000     0   1.000000  14.0
 11   [2, 1]        chemist.png        0.861     3   0.999986   5.0
 12   [2, 2]  knight_statue.png        0.844     1   1.000000   

unit              rank
demon_hunter.png  1       1
dryad.png         1       1
Name: unit, dtype: int64
unit       rank
dryad.png  1       1
Name: unit, dtype: int64 unit              rank
demon_hunter.png  1       1
Name: unit, dtype: int64 unit              rank
demon_hunter.png  1       1
dryad.png         1       1
Name: unit, dtype: int64


## Quick restart

In [32]:
# Drop co-op game
bot.restart_RR(quick_disconnect=True)

In [ ]:
# Restart game
bot.restart_RR()